# Analysis: track ring eigenvectors

In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import proplot as pplt
import psdist as ps
import psdist.visualization as psv
import yaml
from ipywidgets import interact
from ipywidgets import widgets
from omegaconf import OmegaConf
from omegaconf import DictConfig
from pprint import pprint

In [ ]:
pplt.rc["cmap.discrete"] = False
pplt.rc["cmap.sequential"] = "viridis"
pplt.rc["cycle"] = "538"
pplt.rc["grid"] = False
pplt.rc["figure.facecolor"] = "white"

## Setup

In [ ]:
timestamp = None
script_name = "track_eig"

if timestamp is None:
    input_dirs = os.listdir(f"./outputs/{script_name}")
    input_dirs = sorted(input_dirs)
    input_dir = input_dirs[-1]
    input_dir = os.path.join(f"./outputs/{script_name}", input_dir)
else:
    input_dir = f"./outputs/{script_name}/{timestamp}/"

print("intput_dir = ", input_dir)

In [ ]:
cfg_path = os.path.join(input_dir, "config/config.yaml")
cfg = yaml.safe_load(open(cfg_path, "r"))
cfg = DictConfig(cfg)

print("config:")
print(OmegaConf.to_yaml(cfg))

## Load transfer matrix

In [ ]:
M = np.loadtxt(os.path.join(input_dir, "transfer_matrix.dat"))

from orbitsim import coupling
eigvals, eigvecs = np.linalg.eig(M)
eigtunes = coupling.eigentunes_from_eigenvalues(eigvals)
eigvecs = coupling.normalize_eigenvectors(eigvecs)

## Analysis

In [ ]:
coords = np.load(os.path.join(input_dir, "coords.npy"))
coords = coords[:, :, :4]
coords *= 1000.0
coords.shape

In [ ]:
v1 = coords[:, 0, :]
v2 = coords[:, 1, :]

### Corner

In [ ]:
dims = ["x", "xp", "y", "yp", "z", "dE"]
units = ["mm", "mrad", "mm", "mrad", "m", "MeV"]
labels = [f"{dim} [{unit}]" for dim, unit in zip(dims, units)]
limits = [ps.points.limits(v, zero_center=True, share=[(0, 2), (1, 3)], pad=0.25) for v in [v1, v2]]
limits = psv.combine_limits(limits)

In [ ]:
grid = psv.CornerGrid(d=4, diag=False)
grid.plot_points(v1, kind="scatter", c="red6")
grid.plot_points(v2, kind="scatter", c="blue6")
grid.set_labels(labels)
grid.set_limits(limits)

# Plot an injected particle.
x = np.zeros((1000, 4))
x[0] = [10.0, +0.25, 0.0, 1.0]
for i in range(1, x.shape[0]):
    x[i] = np.matmul(M, x[i - 1])
grid.plot_points(x, kind="scatter", c="black", s=1)

plt.show()